# Import Packages

To begin with we need to import packages to work with, these being:

1. harmonic - for bayesian evidence computation.
2. numpy - for basic math functions.
3. emcee - for MCMC sampling (can be replaced by any preferred sampling package). 
4. functools.partial - for numerical integration. 
5. matplotlib - for data visualisation.


In [ ]:
import numpy as np
import emcee
import matplotlib.pyplot as plt
from matplotlib import cm
from functools import partial
import sys
sys.path.insert(0, '/Users/matt/Downloads/Software/harmonic')
import harmonic as hm
sys.path.append("../examples")
import utils

# Define Bayesian Posterior Function

Now we will need to define the log-posterior function we are interested in. Here we consider the interesting case of the pathological Rosenbrock function, 

$$
f(x) = \sum_{i=1}^{d-1} \bigg [ 100(x_{i+1} - x_{i}^2)^2 + (x_i - 1)^2 \bigg ]
$$
where $d$ is the dimension of the function and the input domain is usually taken to be $x_i \in [-5.0, 10.0] \: \; \forall i = 1, \dots, d$. The Rosenbrock function is difficult in the sense that convergence to the unimodal minimum is difficult. 

To compliment this choice of likelihood we adopt a uniform prior over the covariate support $x_i \in [-5.0, 10.0]$. This may simply be coded as the prior


In [ ]:
def ln_prior_uniform(x, xmin=-10.0, xmax=10.0, ymin=-5.0, ymax=15.0):
    """
    Compute log_e of uniform prior.
    Args: 
        - x: 
            Position at which to evaluate prior.
        - xmin: 
            Uniform prior minimum x edge (first dimension).
        - xmax: 
            Uniform prior maximum x edge (first dimension).
        - ymin: 
            Uniform prior minimum y edge (second dimension).
        - ymax: 
            Uniform prior maximum y edge (second dimension).             
    Returns:
        - double: 
            Value of prior at specified point.
    """
        
    if x[0] >= xmin and x[0] <= xmax and x[1] >= ymin and x[1] <= ymax:        
        return 1.0 / ( (xmax - xmin) * (ymax - ymin) )
    else:
        return 0.0

and the Rosenbrock likelihood 

In [ ]:

def ln_likelihood(x, a=1.0, b=100.0):
    """
    Compute log_e of likelihood defined by Rosenbrock function.
    Args: 
        - x: 
            Position at which to evaluate likelihood.
        - a: 
            First parameter of Rosenbrock function.   
        - b: 
            First parameter of Rosenbrock function. 
    Returns:
        - double: 
            Value of Rosenbrock at specified point.
    """
    
    ndim = x.size

    f = 0.0

    for i_dim in range(ndim-1):
        f += b*(x[i_dim+1]-x[i_dim]**2)**2 + (a-x[i_dim])**2

    return -f

which is combined into the log posterior function given by

In [ ]:
def ln_posterior(x, ln_prior, a=1.0, b=100.0):
    """
    Compute log_e of posterior.
    Args: 
        - x: 
            Position at which to evaluate posterior.
        - a: 
            First parameter of Rosenbrock function.   
        - b: 
            First parameter of Rosenbrock function.
        - ln_prior: 
            Prior function. 
    Returns:
        - double: 
            Posterior at specified point.
    """
    
    ln_L = ln_likelihood(x, a=a, b=b)

    if not np.isfinite(ln_L):
        return -np.inf
    else:
        return ln_prior(x) + ln_L

for hyper-parameters and b.

# Compute samples using Emcee

In typical fashion we now need to collect some samples of the posterior using whichever MCMC package you wish to use. Here we'll collect samples using the [emcee](https://emcee.readthedocs.io/en/stable/) package.

First we will need to define and initialize some variables:

In [ ]:
# Define parameters for emcee sampling
ndim = 2                    # number of dimensions
nchains = 200               # total number of chains to compute
samples_per_chain = 5000    # number of samples per chain
nburn = 2000                # number of samples to discard as burn in

# Initialize random seed
np.random.seed(10)

# Rosenbrock hyper-parameters
a = 1.0
b = 100.0

# Define ln_prior function
xmin = -10.0
xmax = 10.0
ymin = -5.0
ymax = 15.0  
ln_prior = partial(ln_prior_uniform, xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax)  

Now we need to run the sampler to collect samples:

In [ ]:
# Set initial random position and state
pos = np.random.rand(ndim * nchains).reshape((nchains, ndim)) * 0.1   
rstate = np.random.get_state()

# Instantiate and execute Sampler 
sampler = emcee.EnsembleSampler(nchains, ndim, ln_posterior, args=[ln_prior, a, b])
(pos, prob, state) = sampler.run_mcmc(pos, samples_per_chain, rstate0=rstate) 

# Collect samples into contiguous numpy arrays (discarding burn in)
samples = np.ascontiguousarray(sampler.chain[:,nburn:,:])
lnprob = np.ascontiguousarray(sampler.lnprobability[:,nburn:])

# Harmonic Code

Here is where one would normally start (*i.e.* already with chains ready to compute the evidence).

## Collating samples using harmonic.chains class

Now we simply need to configure the chains into a harmonic friendly shape which we do by:

In [ ]:
# Instantiate harmonic's chains class 
chains = hm.Chains(ndim)
chains.add_chains_3d(samples, lnprob)

# Split the chains into the ones which will be used to train/test the machine learning model
chains_train, chains_test = hm.utils.split_data(chains, training_proportion=0.5)

## Train the machine learning model

Now take the chains_train from the above code and use them to train the model, here we select the Kernel Density estimate. 

First we must define the model parameters

In [ ]:
# Define the model hyper-parameters and domain
nfold = 2
nhyper = 2
step = -2
domain = []
hyper_parameters = [[10**(R)] for R in range(-nhyper+step,step)] 

Now we wish to optimize the hyper-parameters over the domain. We do this *via* cross-validation

In [ ]:
validation_variances = \
            hm.utils.cross_validation(
                    chains_train, \
                    domain, \
                    hyper_parameters, \
                    nfold=nfold, \
                    modelClass=hm.model.KernelDensityEstimate, \
                    verbose=False, \
                    seed=0)

best_hyper_param_ind = np.argmin(validation_variances)
best_hyper_param = hyper_parameters[best_hyper_param_ind]

Now we simply instantiate the model and train it using the optimized hyper-parameters and the training chains generated previously

In [ ]:
model = hm.model.KernelDensityEstimate(ndim, 
                                       domain, 
                                       hyper_parameters=best_hyper_param)
fit_success = model.fit(chains_train.samples, chains_train.ln_posterior)

## Compute the Bayesian evidence

Finally we simply compute the Learnt Harmonic Mean estimator by:

In [ ]:
# Instantiate harmonic's evidence class
ev = hm.Evidence(chains_test.nchains, model)

# Pass the evidence class the test chains and compute the evidence!
ev.add_chains(chains_test)
ln_evidence, ln_evidence_std = ev.compute_ln_evidence()

## Numerical Integration

Now for comparison we quickly compute a brute force numerical integration over the posterior for comparison

In [ ]:
ln_posterior_func = partial(ln_posterior, ln_prior=ln_prior, a=a, b=b)
ln_posterior_grid, x_grid, y_grid = utils.eval_func_on_grid(
                                        ln_posterior_func, 
                                        xmin=-10.0, xmax=10.0, 
                                        ymin=-5.0, ymax=15.0, 
                                        nx=1000, ny=1000)
dx = x_grid[0,1] - x_grid[0,0]
dy = y_grid[1,0] - y_grid[0,0]
evidence_numerical = np.sum(np.exp(ln_posterior_grid))*dx*dy
ln_evidence_numerical = np.log(evidence_numerical)

### Posterior plot

Out of interest why don't we plot the posterior from these samples to see what we're working with!

In [ ]:
ax = utils.plot_surface(ln_posterior_grid, x_grid, y_grid, 
                        samples[0,:,:].reshape((-1, ndim)), 
                        lnprob[0,:].reshape((-1, 1)))              
ax.set_zlabel(r'$\log \mathcal{L}$')
ax.view_init(30,100)
plt.show(block=False)  

with 2D projection

In [ ]:
# Plot posterior image.
ax = utils.plot_image(np.exp(ln_posterior_grid), x_grid, y_grid, 
                      samples.reshape((-1,ndim)),
                      colorbar_label=r'$\mathcal{L}$')

## Error analysis

Lets take a look at what the error on our estimate is!

In [ ]:
print('Numerical integration value: {} || Learnt Harmonic Mean value: {}'.format(
    ln_evidence_numerical, ln_evidence))

# Compare to the learnt harmonic mean estimate 
abs_error = np.abs(ln_evidence_numerical - ln_evidence)
percent_error = 100.0 * abs_error / ln_evidence_numerical

# Compare the variance provided by the learnt harmonic mean estimate
sigma_error = np.log(np.exp(ln_evidence) + np.exp(ln_evidence_std)) - ln_evidence

sigma_count = abs_error / sigma_error

print('Number of sigma away from true value: {}'.format(sigma_count))

# N.B: sigma_count represents the number of standard deviations between the estimated 
#     and analytic results.